In [1]:
import src.data as data

data.DataUpdater.main()
data.DataProcessor.main(True)

Update Files
Tue May 21 06:47:28 2024 : Updated ~ data\DataBase\DB_information\calendar.feather Done! Cost 0.06 Secs
Tue May 21 06:47:28 2024 : Updated ~ data\DataBase\DB_information\description.feather Done! Cost 0.04 Secs
Tue May 21 06:47:28 2024 : Updated ~ data\DataBase\DB_information\st.feather Done! Cost 0.02 Secs
Tue May 21 06:47:30 2024 : Updated ~ data\DataBase\DB_information\industry.feather Done! Cost 2.32 Secs
Tue May 21 06:47:35 2024 : Updated ~ data\DataBase\DB_information\concepts.feather Done! Cost 4.35 Secs
Tue May 21 06:47:36 2024 : Updated ~ data\DataBase\DB_models\longcl_exp\2024\longcl_exp.20240517.feather Done! Cost 0.08 Secs
Tue May 21 06:47:36 2024 : Updated ~ data\DataBase\DB_models\risk_exp\2024\risk_exp.20240520.feather Done! Cost 0.05 Secs
Tue May 21 06:47:38 2024 : Updated ~ data\DataBase\DB_trade\day\2024\day.20240520.feather Done! Cost 0.05 Secs
Tue May 21 06:47:38 2024 : Updated ~ data\DataBase\DB_trade\5day\2024\5day.20240520.feather Done! Cost 0.24 Sec

In [1]:
import src.data as data

data.DataProcessor.main(True)

Use device name: NVIDIA GeForce RTX 4090
predict is True , Data Processing start!
4 datas :['y', 'day', '30m', 'risk']
y blocks loading start!
 --> labels blocks reading [ret10_lag] DataBase...... finished! Cost 0.79 secs
 --> labels blocks reading [ret20_lag] DataBase...... finished! Cost 0.65 secs
 --> labels blocks merging (2)...... finished! Cost 0.09 secs
 --> models blocks reading [risk_exp] DataBase...... finished! Cost 3.24 secs
y blocks loading finished! Cost 5.17 secs
y blocks process...... finished! Cost 2.08 secs
y blocks masking...... finished! Cost 0.06 secs
y blocks saving ...... finished! Cost 0.17 secs
y blocks norming...... finished! Cost 0.00 secs
y finished! Cost 7.59 Seconds
--------------------------------------------------------------------------------
day blocks loading start!
 --> trade blocks reading [day] DataBase...... finished! Cost 1.37 secs
day blocks loading finished! Cost 1.37 secs
day blocks process...... finished! Cost 0.11 secs
day blocks masking....

In [1]:
from src.interface import Predictor

model_preds = [
    Predictor('gru_day'    , 'swalast' , 0 , 'gru_day_V0') ,
    Predictor('gruRTN_day' , 'swalast' , 0 , 'gruRTN_day_V0') ,
    Predictor('gruRES_day' , 'swalast' , 0 , 'gruRES_day_V0')
]
[md.get_df().deploy() for md in model_preds]

Load  2 DataBlocks...... finished! Cost 0.10 secs
Align 2 DataBlocks...... finished! Cost 0.17 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
20231201 new


FileNotFoundError: [WinError 53] 找不到网络路径。: '//hfm-pubshare/HFM各部门共享/'

In [1]:
import torch
d = torch.load('./model/gruRES_day/0/20170103/best/state_dict.pt')
d

OrderedDict([('multiloss_alpha', tensor([0.9477])),
             ('encoder.mod_list.0.fc_enc_in.0.weight',
              tensor([[-2.5078e-01,  1.3785e-01, -3.3354e-02, -6.8577e-02, -5.5988e-02,
                       -1.7658e-01],
                      [-3.5977e-01, -6.8289e-02,  1.3821e-01,  4.0638e-01, -1.9340e-02,
                       -2.2752e-01],
                      [ 2.1029e-02, -3.6252e-01,  9.2285e-02, -5.8660e-02, -8.9976e-02,
                        1.5672e-02],
                      [-2.7325e-02,  1.8681e-02,  1.2082e-01,  1.2036e-01, -3.2528e-02,
                       -3.0512e-01],
                      [ 1.6647e-01, -1.0239e-02, -3.9088e-01, -1.9459e-01,  2.0365e-01,
                        4.2245e-01],
                      [ 2.6572e-01, -2.2931e-01, -5.3139e-01,  4.7760e-01,  4.7605e-01,
                        3.2853e-02],
                      [-1.4737e-01,  2.0000e-01,  5.0303e-01, -3.1841e-01, -5.6933e-03,
                        4.4064e-02],
                  

In [1]:
import src.data as data
data.DataProcessor.main(True)

predict is True , Data Processing start!
3 datas :['y', 'day', '30m']
y blocks loading start!
 --> labels blocks reading [ret10_lag] DataBase's...... finished! Cost 1.03 secs
 --> labels blocks reading [ret20_lag] DataBase's...... finished! Cost 0.92 secs
 --> labels blocks merging...... finished! Cost 0.10 secs
 --> models blocks reading [risk_exp] DataBase's...... finished! Cost 3.99 secs
 --> models blocks merging...... finished! Cost 0.00 secs
y blocks loading finished! Cost 6.53 secs
y blocks process...... finished! Cost 4.47 secs
y blocks masking...... finished! Cost 0.08 secs
y blocks saving ...... finished! Cost 0.14 secs
y blocks norming...... finished! Cost 0.00 secs
y finished! Cost 11.29 Seconds
--------------------------------------------------------------------------------
day blocks loading start!
 --> trade blocks reading [day] DataBase's...... finished! Cost 1.52 secs
 --> trade blocks merging...... finished! Cost 0.00 secs
day blocks loading finished! Cost 1.55 secs
d

In [1]:
import src.data as data
data.DataProcessor.main(False)

Use device name: NVIDIA GeForce RTX 4090
predict is False , Data Processing start!
4 datas :['y', 'day', '30m', 'risk']
y blocks loading start!
 --> labels blocks reading [ret10_lag] DataBase...... finished! Cost 16.02 secs
 --> labels blocks reading [ret20_lag] DataBase...... finished! Cost 15.67 secs
 --> labels blocks merging (2)...... finished! Cost 2.87 secs
 --> models blocks reading [risk_exp] DataBase...... finished! Cost 61.10 secs
y blocks loading finished! Cost 106.39 secs
y blocks process...... finished! Cost 46.00 secs
y blocks masking...... finished! Cost 0.73 secs
y blocks saving ...... finished! Cost 3.66 secs
y blocks norming...... finished! Cost 0.00 secs
y finished! Cost 156.92 Seconds
--------------------------------------------------------------------------------
day blocks loading start!
 --> trade blocks reading [day] DataBase...... finished! Cost 28.16 secs
day blocks loading finished! Cost 28.19 secs
day blocks process...... finished! Cost 3.32 secs
day blocks 

In [2]:
from src.interface import ModelTestor

mt = ModelTestor.new('ts_mixer')
mt.try_metrics()

Load  2 DataBlocks ...

... cost 0.22 secs
Align 2 DataBlocks ...... cost 0.21 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
x shape is torch.Size([5064, 30, 6])
y shape is torch.Size([5064, 1])
Test Forward Success
metrics :  Metrics.MetricOutput(loss=tensor(1.0688, grad_fn=<AddBackward0>), score=-0.015420470386743546, loss_item=1.068833589553833, penalty=0.0, losses=tensor(1.0688, grad_fn=<ExpBackward0>))
Test Metrics Success


In [2]:
from src.data.process import select_processor
select_processor('day')